## Health Metrics - Data Cleaning

### Initialize packages

In [2]:
import pandas as pd

### Import data into frames

In [17]:
data = pd.read_json('data.json')["data"]
workouts_df = pd.DataFrame(data["workouts"])
metrics_df = pd.DataFrame(data["metrics"])

,id,name,start,end,duration,activeEnergyBurned,intensity,distance,location,humidity,temperature,lapLength
0,1A92CCA1-80B0-4C44-9FA5-DD5A2756478C,Core Training,2021-03-11 07:03:11 -1000,2021-03-11 07:14:18 -1000,636.608887,"{'units': 'kcal', 'qty': 24.111050124606475}","{'units': 'kcal/hr·kg', 'qty': 3.2253185291250...",NaN,NaN,NaN,NaN,NaN
1,BAD3330A-E7F0-4C54-9257-43075838357D,High Intensity Interval Training,2021-03-24 14:44:10 -1000,2021-03-24 14:51:11 -1000,421.000305,"{'units': 'kcal', 'qty': 62.307994144769985}","{'units': 'kcal/hr·kg', 'qty': 8.19168350418671}",NaN,NaN,NaN,NaN,NaN
2,00DA3796-BA60-42EB-B28A-74AD67C5C1B5,Outdoor Cycling,2021-03-26 03:31:24 -1000,2021-03-26 03:40:00 -1000,515.888359,"{'units': 'kcal', 'qty': 2.2510000000000012}","{'units': 'kcal/hr·kg', 'qty': 2.404163307713482}","{'units': 'mi', 'qty': 0.251547935379472}",Outdoor,NaN,NaN,NaN
3,AC234210-6900-45E3-A39F-7EF6505914FE,High Intensity Interval Training,2021-04-01 11:30:19 -1000,2021-04-01 11:54:35 -1000,948.742759,"{'units': 'kcal', 'qty': 104.66971473098202}","{'units': 'kcal/hr·kg', 'qty': 6.361804009297137}",NaN,NaN,NaN,NaN,NaN
4,2FE239D9-3311-4C24-8A4B-C876EEAC64FB,High Intensity Interval Training,2021-04-06 10:28:53 -1000,2021-04-06 10:34:36 -1000,5.955130,"{'units': 'kcal', 'qty': 0.09641972280982501}",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
397,9C284EEF-9DC7-4DDE-B922-28E273040FD7,Pool Swim,2024-12-20 15:17:04 -1000,2024-12-20 16:15:14 -1000,3489.925188,"{'units': 'kcal', 'qty': 518.9220455934935}","{'units': 'kcal/hr·kg', 'qty': 7.8493174569585...","{'units': 'mi', 'qty': 1.149536705639067}",Pool,"{'units': '%', 'qty': 72}","{'units': 'degF', 'qty': 79.46599999999562}","{'units': 'mi', 'qty': 0.015534279805933}"
398,02EE2674-9C77-4EF5-8F2F-CC300E0C35A4,Outdoor Walk,2024-12-20 16:23:06 -1000,2024-12-20 17:27:40 -1000,3873.381206,"{'units': 'kcal', 'qty': 250.92600000000036}","{'units': 'kcal/hr·kg', 'qty': 4.145482301834785}","{'units': 'mi', 'qty': 2.401984044172394}",Outdoor,"{'units': '%', 'qty': 75}","{'units': 'degF', 'qty': 78.00799999999563}",NaN
399,8818490E-F8E0-4D79-9CA0-FEC93DD36BB5,Outdoor Walk,2024-12-22 12:24:05 -1000,2024-12-22 13:27:40 -1000,3814.116406,"{'units': 'kcal', 'qty': 273.32}","{'units': 'kcal/hr·kg', 'qty': 4.457631278379929}","{'units': 'mi', 'qty': 2.437099998215137}",Outdoor,"{'units': '%', 'qty': 63}","{'units': 'degF', 'qty': 80.57872772216358}",NaN
400,335BE4F3-9079-42AC-BBB6-BBC6CCDF3579,Pool Swim,2024-12-22 13:32:18 -1000,2024-12-22 14:35:26 -1000,3788.127733,"{'units': 'kcal', 'qty': 523.5542748451978}","{'units': 'kcal/hr·kg', 'qty': 7.411601573116997}","{'units': 'mi', 'qty': 1.149536705639067}",Pool,"{'units': '%', 'qty': 62}","{'units': 'degF', 'qty': 80.4125286102251}","{'units': 'mi', 'qty': 0.015534279805933}"


### Clean the data

In [20]:
workouts_df['activeEnergyBurned_qty'] = workouts_df['activeEnergyBurned'].apply(lambda x: x['qty'] if isinstance(x, dict) else x)
workouts_df

,id,name,start,end,duration,activeEnergyBurned,intensity,distance,location,humidity,temperature,lapLength,activeEnergyBurned_qty
0,1A92CCA1-80B0-4C44-9FA5-DD5A2756478C,Core Training,2021-03-11 07:03:11 -1000,2021-03-11 07:14:18 -1000,636.608887,"{'units': 'kcal', 'qty': 24.111050124606475}","{'units': 'kcal/hr·kg', 'qty': 3.2253185291250...",NaN,NaN,NaN,NaN,NaN,24.111050
1,BAD3330A-E7F0-4C54-9257-43075838357D,High Intensity Interval Training,2021-03-24 14:44:10 -1000,2021-03-24 14:51:11 -1000,421.000305,"{'units': 'kcal', 'qty': 62.307994144769985}","{'units': 'kcal/hr·kg', 'qty': 8.19168350418671}",NaN,NaN,NaN,NaN,NaN,62.307994
2,00DA3796-BA60-42EB-B28A-74AD67C5C1B5,Outdoor Cycling,2021-03-26 03:31:24 -1000,2021-03-26 03:40:00 -1000,515.888359,"{'units': 'kcal', 'qty': 2.2510000000000012}","{'units': 'kcal/hr·kg', 'qty': 2.404163307713482}","{'units': 'mi', 'qty': 0.251547935379472}",Outdoor,NaN,NaN,NaN,2.251000
3,AC234210-6900-45E3-A39F-7EF6505914FE,High Intensity Interval Training,2021-04-01 11:30:19 -1000,2021-04-01 11:54:35 -1000,948.742759,"{'units': 'kcal', 'qty': 104.66971473098202}","{'units': 'kcal/hr·kg', 'qty': 6.361804009297137}",NaN,NaN,NaN,NaN,NaN,104.669715
4,2FE239D9-3311-4C24-8A4B-C876EEAC64FB,High Intensity Interval Training,2021-04-06 10:28:53 -1000,2021-04-06 10:34:36 -1000,5.955130,"{'units': 'kcal', 'qty': 0.09641972280982501}",NaN,NaN,NaN,NaN,NaN,NaN,0.096420
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,9C284EEF-9DC7-4DDE-B922-28E273040FD7,Pool Swim,2024-12-20 15:17:04 -1000,2024-12-20 16:15:14 -1000,3489.925188,"{'units': 'kcal', 'qty': 518.9220455934935}","{'units': 'kcal/hr·kg', 'qty': 7.8493174569585...","{'units': 'mi', 'qty': 1.149536705639067}",Pool,"{'units': '%', 'qty': 72}","{'units': 'degF', 'qty': 79.46599999999562}","{'units': 'mi', 'qty': 0.015534279805933}",518.922046
398,02EE2674-9C77-4EF5-8F2F-CC300E0C35A4,Outdoor Walk,2024-12-20 16:23:06 -1000,2024-12-20 17:27:40 -1000,3873.381206,"{'units': 'kcal', 'qty': 250.92600000000036}","{'units': 'kcal/hr·kg', 'qty': 4.145482301834785}","{'units': 'mi', 'qty': 2.401984044172394}",Outdoor,"{'units': '%', 'qty': 75}","{'units': 'degF', 'qty': 78.00799999999563}",NaN,250.926000
399,8818490E-F8E0-4D79-9CA0-FEC93DD36BB5,Outdoor Walk,2024-12-22 12:24:05 -1000,2024-12-22 13:27:40 -1000,3814.116406,"{'units': 'kcal', 'qty': 273.32}","{'units': 'kcal/hr·kg', 'qty': 4.457631278379929}","{'units': 'mi', 'qty': 2.437099998215137}",Outdoor,"{'units': '%', 'qty': 63}","{'units': 'degF', 'qty': 80.57872772216358}",NaN,273.320000
400,335BE4F3-9079-42AC-BBB6-BBC6CCDF3579,Pool Swim,2024-12-22 13:32:18 -1000,2024-12-22 14:35:26 -1000,3788.127733,"{'units': 'kcal', 'qty': 523.5542748451978}","{'units': 'kcal/hr·kg', 'qty': 7.411601573116997}","{'units': 'mi', 'qty': 1.149536705639067}",Pool,"{'units': '%', 'qty': 62}","{'units': 'degF', 'qty': 80.4125286102251}","{'units': 'mi', 'qty': 0.015534279805933}",523.554275
